In [1]:
from sqlalchemy import create_engine
from db_config import *

# utilities
import re
import pickle
import numpy as np
import pandas as pd

# plotting
import seaborn as sns
import matplotlib.pyplot as plt

# nltk
from nltk.stem import WordNetLemmatizer

# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import plot_roc_curve


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report


MNB = pickle.load(open('/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Classification/Code/saved_models/Sentiment-MNB.pickle', 'rb'))
LR = pickle.load(open('/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Classification/Code/saved_models/Sentiment-LR.pickle', 'rb'))
SVM = pickle.load(open('/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Classification/Code/saved_models/Sentiment-SVC.pickle', 'rb'))
vectorisor = pickle.load(open('/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Classification/Code/saved_models/vectoriser-ngram-(1,2).pickle', 'rb'))

In [2]:
# Defining dictionary containing all emojis with their meanings.
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}
          
## Defining set containing all stopwords in english.
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from', 
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [3]:
def preprocess(textdata):
    processedText = []
    
    # Create Lemmatizer and Stemmer.
    wordLemm = WordNetLemmatizer()
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[^\s]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    for tweet in textdata:
        tweet = tweet.lower()
        
        # Replace all URls with 'URL'
        tweet = re.sub(urlPattern,' URL',tweet)
        # Replace all emojis.
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji])        
        # Replace @USERNAME to 'USER'.
        tweet = re.sub(userPattern,' USER', tweet)        
        # Replace all non alphabets.
        tweet = re.sub(alphaPattern, " ", tweet)
        # Replace 3 or more consecutive letters by 2 letter.
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

        tweetwords = ''
        for word in tweet.split():
            # Checking if the word is a stopword.
            #if word not in stopwordlist:
            if len(word)>1:
                # Lemmatizing the word.
                word = wordLemm.lemmatize(word)
                tweetwords += (word+' ')
            
        processedText.append(tweetwords)
        
    return processedText

In [4]:
table = 'social.blockchain'
classifier_name = 'MNB'
start_date = '2021-01-01'
end_date = '2020-12-31'

stmt = f'SELECT id, text FROM {table} WHERE date >= \'{start_date}\''

engine = create_engine('mysql+mysqlconnector://'+ user + ':' + passwd + '@' + ip + ':3306/' + schema1)
df = pd.read_sql(stmt, engine)

In [5]:
import time
text = df['text']
t = time.time()
processedtext = preprocess(text)
print(f'Text Preprocessing complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

Text Preprocessing complete.
Time Taken: 455 seconds


In [6]:
X  = vectorisor.transform(processedtext)

In [7]:
MNB_pred = MNB.predict(X)
MNB_prob = MNB.predict_proba(X)

LR_pred = LR.predict(X)
LR_prob = LR.predict_proba(X)

SVM_pred = SVM.predict(X)
SVM_prob = SVM.predict_proba(X)

In [8]:
a = LR_pred
b = SVM_pred
c = MNB_pred

d = LR_prob
e = SVM_prob
f = MNB_prob

ensemble_pred = []
ensemble_prob = []

for i in range(len(a)):
    temp = []
    temp2 = []

    temp.append(a[i])
    temp.append(b[i])
    temp.append(c[i])
    ensemble_pred.append(max(set(temp), key=temp.count))

    temp2.append(max(d[i]))
    temp2.append(max(e[i]))
    temp2.append(max(f[i]))
    ensemble_prob.append(np.mean(temp2))

In [9]:
mnb_model_prob = []
for i in range(len(MNB_pred)):
    if MNB_pred[i] == 1:
        mnb_model_prob.append(max(MNB_prob[i]))
    else:
        mnb_model_prob.append(1 - max(MNB_prob[i]))


lr_model_prob = []
for i in range(len(LR_pred)):
    if LR_pred[i] == 1:
        lr_model_prob.append(max(LR_prob[i]))
    else:
        lr_model_prob.append(1 - max(LR_prob[i]))


svm_model_prob = []
for i in range(len(SVM_pred)):
    if SVM_pred[i] == 1:
        svm_model_prob.append(max(SVM_prob[i]))
    else:
        svm_model_prob.append(1 - max(SVM_prob[i]))


In [10]:
bc_sentiment = df.iloc[:,[0]]
bc_sentiment['mnb_prediction'] = MNB_pred
bc_sentiment['mnb_probability'] = np.array(mnb_model_prob)

bc_sentiment['lr_prediction'] = LR_pred
bc_sentiment['lr_probability'] = np.array(lr_model_prob)


bc_sentiment['svm_prediction'] = SVM_pred
bc_sentiment['svm_probability'] = np.array(svm_model_prob)


bc_sentiment['ensemble_prediction'] = np.array(ensemble_pred)
bc_sentiment['ensemble_probability'] = np.array(ensemble_prob)

In [11]:
bc_sentiment

,id,mnb_prediction,mnb_probability,lr_prediction,lr_probability,svm_prediction,svm_probability,ensemble_prediction,ensemble_probability
0,1355667103467114504,1,0.596444,1,0.730653,1,0.583830,1,0.636975
1,1355666949204807682,1,0.763248,1,0.927706,1,0.527601,1,0.739518
2,1355666884042043396,1,0.596444,1,0.730653,1,0.583830,1,0.636975
3,1355666857336872962,1,0.501853,1,0.590999,1,0.563715,1,0.552189
4,1355666806321655816,1,0.596444,1,0.730653,1,0.583830,1,0.636975
...,...,...,...,...,...,...,...,...,...
5066468,1465833023274995724,0,0.477750,1,0.548712,1,0.553264,1,0.541408
5066469,1465833022914129921,1,0.668502,1,0.761130,1,0.538547,1,0.656060
5066470,1465833022662586368,0,0.494745,1,0.648349,1,0.583830,1,0.579145
5066471,1465833021597179916,1,0.600336,1,0.625550,1,0.547851,1,0.591246


In [12]:
bc_sentiment.to_sql('BC_sentiment' , con=engine, if_exists='append', index=False, chunksize=20000)